# Exercises II

# Let's set up an example scenario as perspective for our regression exercises using the Zillow dataset.
--------------------------------------------------------------------------------------------------------------------------------------------------------
# As a Codeup data science graduate, you want to show off your skills to the Zillow data science team in hopes of getting an interview for a position you saw pop up on LinkedIn. You thought it might look impressive to build an end-to-end project in which you use some of their Kaggle data to predict property values using some of their available features; who knows, you might even do some feature engineering to blow them away. Your goal is to predict the values of single unit properties using the obervations from 2017.

# Acquire bedroomcnt, bathroomcnt, calculatedfinishedsquarefeet, taxvaluedollarcnt, yearbuilt, taxamount, and fips from the zillow database for all 'Single Family Residential' properties.

In [4]:
import env
import pandas as pd
def aquire_zillow_data():
    import os 
    import env
   
    query='\
    select bedroomcnt, bathroomcnt, calculatedfinishedsquarefeet, taxvaluedollarcnt, yearbuilt, taxamount,fips\
    from properties_2017\
    left join propertylandusetype using(propertylandusetypeid)\
    where propertylandusedesc = "Single Family Residential"'
    
    if os.path.exists('zillow.csv') == True:
        return pd.read_csv('zillow.csv')
    else:
        df = pd.read_sql(query, get_connection())
        df.to_csv('zillow.csv')
        return pd.read_csv('zillow.csv')
        
def get_connection(user=env.user,password=env.password,host=env.host,database=env.database):
    return f'mysql+pymysql://{user}:{password}@{host}/{database}'      
    

In [8]:
df = aquire_zillow_data()

# In these exercises, you will complete the first step toward the above goal: acquire and prepare the necessary Zillow data from the zillow database in the Codeup database server.

# Using your acquired Zillow data, walk through the summarization and cleaning steps in your wrangle.ipynb file like we did above. You may handle the missing values however you feel is appropriate and meaningful; remember to document your process and decisions using markdown and code commenting where helpful.

In [25]:
# First I want to see what my data looks like. 
df.info()
# My first impression is why is the unnamed on here so im 
# am going to investigate to see if we can drop the column

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2152863 entries, 0 to 2152862
Data columns (total 8 columns):
 #   Column                        Dtype  
---  ------                        -----  
 0   Unnamed: 0                    int64  
 1   bedroomcnt                    float64
 2   bathroomcnt                   float64
 3   calculatedfinishedsquarefeet  float64
 4   taxvaluedollarcnt             float64
 5   yearbuilt                     float64
 6   taxamount                     float64
 7   fips                          float64
dtypes: float64(7), int64(1)
memory usage: 131.4 MB


In [29]:
df.head()
# Its safe to sat that the column matches the index so 
# i can confidently drop the column but for safety measures im also going
# check the last few values
df.tail()

,Unnamed: 0,bedroomcnt,bathroomcnt,calculatedfinishedsquarefeet,taxvaluedollarcnt,yearbuilt,taxamount,fips
2152858,2152858,4.0,3.0,2262.0,960756.0,2015.0,13494.52,6059.0
2152859,2152859,4.0,4.5,3127.0,536061.0,2014.0,6244.16,6059.0
2152860,2152860,0.0,0.0,NaN,208057.0,NaN,5783.88,6059.0
2152861,2152861,3.0,2.5,1974.0,424353.0,2015.0,5302.70,6059.0
2152862,2152862,4.0,4.0,2110.0,554009.0,2014.0,6761.20,6037.0


In [31]:
# we will reassign my data frame 
df = df.drop(columns='Unnamed: 0')

### Now lets deal with nulls !!!!

In [20]:
(df.isna().sum())/len(df) *100
# all the nulls are less than 1% so i can drop them 

Unnamed: 0                      0.000000
bedroomcnt                      0.000511
bathroomcnt                     0.000511
calculatedfinishedsquarefeet    0.394080
taxvaluedollarcnt               0.022900
yearbuilt                       0.433702
taxamount                       0.206330
fips                            0.000000
dtype: float64

In [33]:
df = df.dropna()

In [34]:
(df.isna().sum())/len(df) *100

bedroomcnt                      0.0
bathroomcnt                     0.0
calculatedfinishedsquarefeet    0.0
taxvaluedollarcnt               0.0
yearbuilt                       0.0
taxamount                       0.0
fips                            0.0
dtype: float64

In [ ]:
# now 0% of the cells are empty so 100% of cells are full 

# now lets address those ugly columns names

In [41]:
#first lets make a list of the name of the column names that
# I want to use

cols_name = ['bedrooms', 'bathrooms', 'calculated_finished_squarefeet',
       'tax_valuedollar_cnt', 'year_built', 'tax_amount', 'fips']

df.set_axis(cols_name, axis=1,inplace=True)
df

,bedrooms,bathrooms,calculated_finished_squarefeet,tax_valuedollar_cnt,year_built,tax_amount,fips
4,4.0,2.0,3633.0,296425.0,2005.0,6941.39,6037.0
6,3.0,4.0,1620.0,847770.0,2011.0,10244.94,6037.0
7,3.0,2.0,2077.0,646760.0,1926.0,7924.68,6037.0
11,0.0,0.0,1200.0,5328.0,1972.0,91.60,6037.0
14,0.0,0.0,171.0,6920.0,1973.0,255.17,6037.0
...,...,...,...,...,...,...,...
2152856,4.0,4.0,4375.0,422400.0,2015.0,13877.56,6037.0
2152858,4.0,3.0,2262.0,960756.0,2015.0,13494.52,6059.0
2152859,4.0,4.5,3127.0,536061.0,2014.0,6244.16,6059.0
2152861,3.0,2.5,1974.0,424353.0,2015.0,5302.70,6059.0


Store all of the necessary functions to automate your process from acquiring the data to returning a cleaned dataframe with no missing values in your wrangle.py file. Name your final function wrangle_zillow.